### Random Sample Function in Python

In [5]:
import random
random.sample([1,2,3,4,5,6,7,8,9,10], 3)

[10, 5, 8]

In [7]:
data = random.choices([True, False], weights=[0.1, 0.9], k=100)
data[0:10]

[False, False, False, False, False, False, False, False, True, False]

In [9]:
from collections import Counter
Counter(data)


Counter({False: 93, True: 7})

## Why Reduce Big Data?

* Despite the importance of "big data", sometimes using the entire dataset is infeasible, impractical, or expensive.

1. Storage Limitations: 
   - Many applications can't keep all the data is is collected
   - Example: A large ISP might generate petabytes of log data daily

2. Convenience: 
   - Working with smaller datasets is often more manageable
   - It's easier to experiment and iterate on smaller data

3. Speed: 
   - Analyzing a compact summary is faster than processing the entire dataset



In [ ]:
## Why Sample?

Sampling is a powerful technique for reducing big data. Here's why it's often the go-to method:

1. Intuitive Semantics: 
   - We get a smaller dataset that maintains the structure of the original
   - It's like taking a small scoop from a large pot of soup to taste it

2. Straightforward Estimation: 
   - We can often run the same analysis on the sample as on the full data
   - We just need to apply some rescaling to our results

3. General and Agnostic: 
   - Sampling works for many types of analyses
   - Unlike some summary methods that only work for specific computations

4. Easy to Understand: 
   - Most people have an intuition about how sampling works
   - This makes it easier to explain your methods to non-technical stakeholders


In [ ]:
## Sampling as a Mediator of Constraints

In real-world applications, sampling often acts as a bridge between various constraints:

1. Data Characteristics:
   - Examples: Heavy-tailed distributions, correlations between variables
   - Challenge: These can make simple random sampling less effective

2. Query Requirements:
   - Examples: Need for ad-hoc queries, high accuracy, fast aggregations
   - Challenge: Different requirements may need different sampling strategies

3. Resource Constraints:
   - Examples: Limited bandwidth, storage, or CPU power
   - Challenge: We need to sample in a way that respects these limitations


### Sampling Terminology.

* In statistical language, the full dataset is termed the **population**, while the subset is called a **sample**.
* There are various methods and strategies for sampling.
    *  They all provide a condensed version of the data for efficient processing.



### Sampling

<div align="center">
    <img src="https://www.dropbox.com/s/vzr0w2cvx6b5bm0/sampling.png?dl=1" width="700" alt="Sampling Image">
</div>


### Representing Data 

* For a dataset with D variables, a data point can be visualized as a point in a D-dimensional space.
  * For instance, a dataset with two variables can be represented in two dimensions.
<div align="center">
<img src="https://www.dropbox.com/s/t0ca5t1qzkr635b/2d-data.png?dl=1" alt="Drawing" width="700px;"/>
</div>



### Representing Data  -- Example

<div align="center">
    <img src="https://www.dropbox.com/s/sy00s9t8bq0wbxb/data_2d.png?dl=1" alt="Drawing" width="900px;"/>
</div>

### Representing Data  -- Higher Dimensional Space
<div align="center">
<img src="https://www.dropbox.com/s/4flfaojlfb2a101/3d-data.png?dl=1" alt="Drawing" style="width: 700px;"/>
</div>

### Subsampling

* Subsampling is a specific type of sampling where you take a smaller sample from an already obtained sample rather than from the whole population
  * The terms sampling and subsampling are often used interchangeably

* Different methods can be used to subsample.
  * A simple method might involve selecting the first half of the data and discarding the rest.
    * This is not always effective, especially if the data is organized (i.e., sorted) based on specific attributes, such as gender.
* Several more sophisticated sampling strategies exist to efficiently reduce dataset size, especially for big data.


### Random Subsample
* Random subsampling is a method returns random sub-samples from a collection.
* Each item has an equal chance of being selected.
* Best suited for datasets that:
  * Have relatively balanced category proportions.
  * Don't need to specifically account for or analyze the edge cases or outliers in the dataset

In [ ]:
# Bernoulli Sampling

- **Simplest case**: All weights are 1
  - N objects, sample k uniformly
  - Each k-subset equally likely

- **Method**: Sample index from N (no replacement) k times
  - Note: Generating true random numbers on computers?
  - Assume random number generators are good enough



In [10]:
import random
random.sample([1,2,3,4,5], 2)

[3, 1]

In [ ]:
## Bernouilli Sampling
- **Streaming Challenge**: Single linear scan to draw sample
  - Streaming computation: see each element once
  - Application: Streaming sensor data.
  - Common tech interview question

In [12]:
import random

def reservoir_sample(stream, k):
    reservoir = []
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            # Randomly decide whether to include the current item in the reservoir
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir

stream = range(1000) 
sample_size = 10
result = reservoir_sample(stream, sample_size)
print(f"Sample of {sample_size} items: {result}")

Sample of 10 items: [267, 518, 899, 27, 77, 925, 415, 395, 83, 521]


In [ ]:
## Order Sampling

* Also referred to bottom-k sampling
  * Alternative method for reservoir sampling
* Approach 
  * We generate a random tag (a float between 0 and 1) for the item.
  * If we haven't filled our reservoir (heap) to size k yet, we add the item to the heap.
  * If our heap is full, we compare the new item's tag to the largest tag in the heap (which is at the root of the max heap).
  * If the new tag is smaller, we remove the item with the largest tag and add the new item.

* It processes the stream in a single pass, using O(k) additional memory.
* It's easily parallelizable. You can run this on multiple streams and then merge the results by keeping the k items with the smallest tags across all streams.
* It's efficient, with O(log k) time complexity per item due to heap operations.

In [13]:
import heapq
import random

def order_sampling(stream, k):
    heap = []
    
    for item in stream:
        tag = random.random()
        
        if len(heap) < k:
            heapq.heappush(heap, (-tag, item))
        elif -tag > heap[0][0]:
            # If the new item's tag is smaller than the largest in the heap,
            # remove the largest and add the new item
            heapq.heappop(heap)
            heapq.heappush(heap, (-tag, item))
    
    # Return the items (without their tags) in arbitrary order
    return [item for _, item in heap]

stream = range(1000) 
sample_size = 10
result = order_sampling(stream, sample_size)
print(f"Sample of {sample_size} items: {result}")

Sample of 10 items: [86, 576, 742, 572, 952, 669, 329, 195, 469, 956]


### Special Cases

* While random subsampling treats data points equally, there are scenarios where our data has more complex structures or patterns.
  * For example, We might want our sample to mirror certain characteristics of the full dataset or ensure specific segments are represented.

* For this, we turn to subsampling, a broader umbrella term that encompasses various methods including random and stratified subsampling.

In [ ]:
## Weighted Random Sampling
* This method generalizes min-wise sampling and can be proven to provide the correct sampling distribution.
* For each item, draw a random number $r_n$ uniformly from [0,1].
* Compute the 'tag' of an item as $r_n^{(1/x_n)}$, where $x_n$ is the item's weight.
* Keep the k items with the smallest tags.


## Weighted Random Sampling -- Example

Item A: weight 3
Item B: weight 1
Item C: weight 2
Item D: weight 4
Step 1: Generate random numbers and calculate tags
A: r_A = 0.6, tag_A = 0.6^(1/3) ≈ 0.843
B: r_B = 0.2, tag_B = 0.2^(1/1) = 0.200
C: r_C = 0.8, tag_C = 0.8^(1/2) ≈ 0.894
D: r_D = 0.5, tag_D = 0.5^(1/4) ≈ 0.841
Step 2: Keep track of the 2 largest tags
After A: [A(0.843)]
After B: [A(0.843), B(0.200)]
After C: [C(0.894), A(0.843)]
After D: [C(0.894), A(0.843)] (D's tag is smaller than A's, so it's not included)

In [ ]:
# Priority Sampling

- Each item $x_i$ given priority $z_i = $x_i / ri_$ with rn uniform random in (0,1)

- Maintain reservoir of $k+1$ items ($x)i$, $z_i$) of highest priority

- Estimation
  - Let z* = (k+1)st highest priority
  - Top-k priority items: weight estimate x'i = max{ xi, z* }
  - All other items: weight estimate zero

- Statistics and bounds
  - x'i unbiased; zero covariance: Cov[x'i, x'j] = 0 for i≠j
  - Relative variance for any subset sum ≤ 1/(k-1) [Szegedy, 2006]

# Priority Sampling - Example

* Step 1: Generate random numbers and calculate priorities
Item A: weight 15, rA = 0.3, zA = 15 / 0.3 = 50
Item B: weight 8,  rB = 0.1, zB = 8 / 0.1 = 80
Item C: weight 12, rC = 0.4, zC = 12 / 0.4 = 30
Item D: weight 20, rD = 0.8, zD = 20 / 0.8 = 25
Item E: weight 6,  rE = 0.05, zE = 6 / 0.05 = 120
Item F: weight 18, rF = 0.6, zF = 18 / 0.6 = 30
Item G: weight 10, rG = 0.2, zG = 10 / 0.2 = 50
Item H: weight 5,  rH = 0.25, zH = 5 / 0.25 = 20


* Step 2: Maintain reservoir of k+1 (4) items with highest priorities
After A: [(A, 50)]
After B: [(B, 80), (A, 50)]
After C: [(B, 80), (A, 50), (C, 30)]
After D: [(B, 80), (A, 50), (C, 30), (D, 25)]
After E: [(E, 120), (B, 80), (A, 50), (C, 30)]
After F: [(E, 120), (B, 80), (A, 50), (F, 30)]
After G: [(E, 120), (B, 80), (G, 50), (A, 50)]
After H: [(E, 120), (B, 80), (G, 50), (A, 50)] (H's priority is lower, so it's not included)

* Step 3: Estimation
z* (4th highest priority) = 50
Weight estimates:
E: x'E = max(6, 50) = 50
B: x'B = max(8, 50) = 50
G: x'G = max(10, 50) = 50
A: x'A = max(15, 50) = 50
C: x'C = 0 (not in top-k)
D: x'D = 0 (not in top-k)
F: x'F = 0 (not in top-k)
H: x'H = 0 (not in top-k)

### Stratified Random Sampling

* Utilized when data can be associated with specific subgroups.
* It ensures that the sample retains the distribution of values for specific features present in the entire population.
* This approach maintains the population's structure by:
  1. Divides the population into homogeneous subgroups or strata.
  2. Selects elements from each stratum in proportion to the stratum's size.



### Subsampling from Labeled Data

* Unlabeled Data: If data doesn't have predefined categories or strata, how do you ensure a representative sample?

* For such data, we can create 'hypothetical' groups or strata using clustering.

* Clustering works by grouping data points based on how similar they are, ensuring that members of one group are more alike than those in other groups.
  * Explicit Selection: Sometimes, we need a sample from each category or group.


### Stratified Random Sampling - Cont'd

* Instances in the population are first divided into homogenous subgroups known as strata
* The number of elements selected in each stratum is proportional to the size of the stratum
  * Thus maintaining the structure


### Representation of an Observation

* Recall that data points can be represented as a point in high-dimensional space

<div align="center">
<img src="https://www.dropbox.com/s/jbriqx9i7jyloyl/data_clusts_1.png?dl=1" alt="drawing" width="500"/>
</div>



### Identifying Clusters
<div align="center">
<img src="https://www.dropbox.com/s/jb81jew9hyf44pp/data_clusts_2.png?dl=1" alt="drawing" width="500"/>
</div>



### Cluster-Based Sampling 
* Cluster before vs. Cluster after Sampling.
<div align="center">
    <img src="https://www.dropbox.com/s/bkd32khr1q8dfhx/before_after.png?dl=1" alt="drawing" width="500"/>
</div>

### Cluster-Based Sampling: Kernel-Based Importance Sampling 

* How can select instances from a cluster?
  * Which ones should we pick?

* Select with probability inversely proportional to the distance of the cluster center

<div align="center">
<img src="https://www.dropbox.com/s/ncrusv71mylkat8/dist_to_center.png?dl=1" alt="drawing" width="700"/>
</div>

### Cluster-Based Sampling:  Weighted Density Sampling

* Weighted density sampling is a method used to select (or discard) events based on their local density.
  * The probability that a data point will be sampled is determined by its local density; that is, the density around that specific data point.
* Local density is defined as the number of data points within a given radius of that point.
* The radius used is typically defined by the user.


### Cluster-Based Sampling

* Advantages:
  * Efficient once clusters are defined.
  * Can be adjusted to pick points close center or more representative of periphery.

* Limitations:
  * Determining the number of clusters can be challenging.
  * The presence of outliers or noisy observations can skew clustering.
  * Might distort the shape of clusters, complicating subsequent analysis.

### Grid-based Sampling

* How can we sample from this?
  * No clear clusters
    
![](https://www.dropbox.com/scl/fi/b2nytya0izd1g1jbxnwre/map.png?rlkey=qhdprfbd2yqutt5q28lm3fgne&dl=1)


### Grid-based Sampling

![](https://www.dropbox.com/scl/fi/tfc8wr3ju8unzm6sp2r7i/grid_on_map.png?rlkey=yw2fvr65eltophw1tsha4yd1s&dl=1)

### Grid-based Sampling

* Divide the data into grid cells and select a number of points per cell
  * It is possible to choose a fixed number of points or to select points based on some cell-specific criteria

* Provides an organized way of ensuring that samples are spread out across the data space
  * Especially important when the distribution of data points is uneven.

* Ideal for spatial and Geospatial Data
![](https://www.dropbox.com/s/nijshpr93rth8fn/grid.png?dl=1)

###  Grid-based Sampling
* Advantages:
  * Ensures that the entire data space is uniformly represented.
    * Mitigate the risk of sampling bias, especially in areas of sparse data,   
  * Can take either a fixed number of points from each cell or use cell-specific criterion
  * The grid is natural stratification of the data space
    
* Limitations:  
  * The size and shape of the grid cells can influence the sampling results.
    * Too large a grid might miss localized patterns, while too small a grid might lead to over-representation of noisy patterns.
  * Can be computationally intenstive, especially for high-dimensional data. 


### Grid-based Sampling Using Random Projections: An intuition

* Step 1: Given some randomly selected line that passes at the origin, project a point that define Vector A so that the projection is perpendicular to that line and new projection is represented uing vector A'
<div align="center">
    <img src="https://www.dropbox.com/s/wdiqdrphg0wzhmi/a_b_projected.png?dl=1" alt="drawing" width="400"/>
</div>

### Grid-based Sampling Using Random Projections: An intuition

* Step 2: select a vector that starts at the oringin and is embedded in the selected lines and normalize by the vector's magnitude.



<img src="https://www.dropbox.com/s/bqaeehgtvm9c9ga/a_b_line.png?dl=1" alt="drawing" width="800"/>


### Finding Orthogonal Vectors

* Step 3: Repeat using an orthogonal (or perpendicular) vetor.

* To find a vector that is orthogonal  to a given vector, you need a vector whose dot product with the given vector is zero.

* Given vector $ \mathbf{v} = \begin{bmatrix} 5 \\ 2 \end{bmatrix} $.

To find a vector $ \mathbf{u} = \begin{bmatrix} x \\ y \end{bmatrix} $ that is orthogonal to $ \mathbf{v} $, the dot product $ \mathbf{u} \cdot \mathbf{v} $ must be zero:

$$ 
\mathbf{u} \cdot \mathbf{v} = 5x + 2y = 0 
$$



### Finding Orthogonal Vectors -- cont'd

* One way to find an orthogonal vector is to swap the components and negate one of them. Using this method:

* If $ \mathbf{v} = \begin{bmatrix} a \\ b \end{bmatrix} $, Then an orthogonal $ \mathbf{u} $ can be $ \begin{bmatrix} -b \\ a \end{bmatrix} $.


* There are infinitely many vectors orthogonal to $ \mathbf{v} $, but $ \begin{bmatrix} -2 \\ 5 \end{bmatrix} $ is one of the simplest ones.   * You could scale this vector by any scalar, and it would still be orthogonal to $ \mathbf{v} $.


### Project on New Orthogonal Vectors

<img src="https://www.dropbox.com/s/wf0nh45do2193i8/a_b_vec_2.png?dl=1" alt="drawing" width="800"/>


### Convert to New Space (Grid)


<img src="https://www.dropbox.com/s/ggohljkopdi6l6n/a_b_new_space.png?dl=1" alt="drawing" width="800"/>



## Similarity Preserving Projection Based Approach

* Pros:
  * Offers theoretical guarantees that points close in high-dimensional space will be assigned to the same grid elements (buckets).
    * Reference: Johnson–Lindenstrauss lemma
  * Computationally efficient.
  * Easily and swiftly computed on GPUs.
  * Effective with wide data sets.

* Cons:
  * Probabilistic nature results in an infinite number of potential grids to select from.
  * Although a 2-D grid is chosen, grids in a much higher-dimensional space may be required based on the data's dimensionality.
  * Assessing the quality of resulting bins can be labor-intensive.

### Projection as a Dot Product

* How do you project a point onto a new axis?
 
* The dot product (or inner product) of two vectors provides the projection of one vector onto the line spanned by the other.

* This projection describes the vector in terms of the reference vector.
  * After normalization, the quantity represents the magnitude of the projected vector relative to the reference vector.

$$
\text{Proj}_{B}A = \frac{A \cdot B}{|B|}
$$
where $A \cdot B$ is the dot product of vectors A and B.

$$
A \cdot B = \sum_{i=1}^{n} A_i \times B_i 
$$

Here, $n$ is the dimension of vectors $A$ and $B$. $|B|$ denotes the magnitude of vector $B$, and it's computed as:

$$
|B| = \sqrt{\sum B_i^2}
$$

The numerator is used to normalize the resulting quantity, expressing it in terms of vector B.



In [12]:
import math

# compute the normalized projection of A onto B.
A = (3,4)
B = (5,2)

A_dot_B = A[0]*B[0] + A[1]*B[1]
amp_B = math.sqrt(B[0]**2 + B[1]**2)
print(f"The magnitude of B is: {amp_B}")
proj = A_dot_B / amp_B
print (f"The magnitude of the projection  {proj}")

print(f"The ratio of the projection in terms of B is {proj/amp_B}")

The magnitude of B is: 5.385164807134504
The magnitude of the projection  4.270992778072193
The ratio of the projection in terms of B is 0.7931034482758621



<img src="https://www.dropbox.com/s/ggohljkopdi6l6n/a_b_new_space.png?dl=1" alt="drawing" width="800"/>


In [13]:
### Projection as a Dot Product

A = (0.95, 8)
B = (5,7)
C = (6,5)
D = (4,2)

amp_D = math.sqrt(D[0]**2 + D[1]**2)
print(f"The magnitude of D is: {amp_D}\n")


A_dot_D = A[0]*D[0] + A[1]*D[1]
proj_A = A_dot_D / amp_D
print (f"The magnitude of the projection  {proj_A}")
print(f"The ratio of the projection in terms of D is {proj_A/amp_D}")
print(f"Projection occurs in bin {math.ceil(proj_A/amp_D)} \n")


B_dot_D = B[0]*D[0] + B[1]*D[1]
proj_B = B_dot_D / amp_D
print (f"The magnitude of the projection  {proj_B}")
print(f"The ratio of the projection in terms of D is {proj_B/amp_D}")
print(f"Projection occurs in bin {math.ceil(proj_B/amp_D)} \n")


C_dot_D = C[0]*D[0] + B[1]*D[1]
proj_C = C_dot_D / amp_D
print (f"The magnitude of the projection  {proj_C}")
print(f"The ratio of the projection in terms of D is {proj_C/amp_D}")
print(f"Projection occurs in bin {math.ceil(proj_C/amp_D)} \n")


The magnitude of D is: 4.47213595499958

The magnitude of the projection  4.427414595449584
The ratio of the projection in terms of D is 0.99
Projection occurs in bin 1 

The magnitude of the projection  7.602631123499284
The ratio of the projection in terms of D is 1.6999999999999997
Projection occurs in bin 2 

The magnitude of the projection  8.497058314499201
The ratio of the projection in terms of D is 1.9
Projection occurs in bin 2 

